In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep  1 16:38:33 2018

@author: jinlong
"""

import os

import zipfile
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = './tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)
print("pre_trained_model.layers:",pre_trained_model.layers )
print("pre_trained_model.layers size:",np.size(pre_trained_model.layers) )
#include_top=False argument, we load a network that 
#doesn't include the classification layers 
#at the top—ideal for feature extraction.
for layer in pre_trained_model.layers:
  layer.trainable = False
  
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output



pre_trained_model.layers: [<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001AC795B5320>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001AC795E0390>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001AC7956FBE0>, <tensorflow.python.keras.layers.core.Activation object at 0x000001AC795247F0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001AC792AFBE0>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001AC8EC32A90>, <tensorflow.python.keras.layers.core.Activation object at 0x000001AC8EC32F60>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001AC6083B4A8>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001AC6087FF98>, <tensorflow.python.keras.layers.core.Activation object at 0x000001AC6087FFD0>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001AC608B2FD0>, <tensorflow.python.keras.l

In [3]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024/32, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])


#local_zip = .'/tmp/cats_and_dogs_filtered.zip'zip_ref = zipfile.ZipFile(local_zip, 'r')#zip_ref.extractall('./tmp')
#zip_ref.close()

# Define our example directories and files
base_dir = './tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)


print("end")
#ResourceExhaustedError: OOM when allocating tensor with shape[3,3,192,320] and type float on
#/job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
#[10,256,400,528]的第一个参数表示batch_size的大小，第二个参数表示某层卷积核的个数，第三个参数表示图像的高，第四个参数表示图像的长

end


In [5]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,#20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,#20,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/2
 - 37s - loss: 0.5008 - acc: 0.7715 - val_loss: 0.1894 - val_acc: 0.9180
Epoch 2/2
 - 33s - loss: 0.3757 - acc: 0.8295 - val_loss: 0.1753 - val_acc: 0.9340
